In [1]:
from flask import Flask, render_template, request, jsonify,url_for
import openai
import pandas as pd
import os
from langchain.agents.agent_types import AgentType
from langchain.agents import load_tools
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_community.chat_models import ChatOpenAI
from langchain.agents.agent_types import *
from langchain_openai import OpenAI
import regex as re
import base64
import shutil
import time

app = Flask(__name__)
openai.api_key = os.environ.get('OPENAI_API_KEY')

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/process', methods=['POST'])
def process():
    start_time=time.time()
    question = request.form.get('question')
    data_source = request.form.get('data_source')
    # print(question)
    print(data_source)
    
    df_BLK = pd.read_csv('C:\CODE\Demo\BLk_DATASET.csv')
    df_blk_meta=pd.read_csv('C:\CODE\Demo\df_blk_meta.csv')
    df_dtc_data=pd.read_csv('C:\CODE\Demo\dtc_dataset.csv')
    
    agent_blk_meta = create_pandas_dataframe_agent(OpenAI(temperature=0), df_blk_meta, verbose=True)

    agent_blk_meta = create_pandas_dataframe_agent(
        ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
        df_blk_meta,
        verbose=True,
        agent_type=AgentType.OPENAI_FUNCTIONS,
    )

    agent_blk = create_pandas_dataframe_agent(OpenAI(temperature=0), df_BLK, verbose=True)

    agent_blk = create_pandas_dataframe_agent(
        ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
        df_BLK,
        verbose=True,
        agent_type=AgentType.OPENAI_FUNCTIONS,
    )
    
    agent_dtc_data= create_pandas_dataframe_agent(OpenAI(temperature=0), df_dtc_data, verbose=True)

    agent_dtc_data= create_pandas_dataframe_agent(
        ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
        df_dtc_data,
        verbose=True,
        agent_type=AgentType.OPENAI_FUNCTIONS,
    )
    
    if data_source == 'metadata':
 
        df_a=agent_blk_meta.run(question)
        print(question)
        # print(df_a)

    elif data_source == 'blk':
        
        df_a=agent_blk.run(question)
        print(question)
    elif data_source == 'dtc':
        
        df_a=agent_dtc_data.run(question)

    else:
        print("Please go back and select a source from the drop down")
    
    print("#########df_a#####\n",df_a)
    
    match = re.search(r'\b[\w-]+\.(?:png|jpg|jpeg|gif)\b', df_a)
    print("here is the match if found:",match)
    
    image_url=None
    if match:
        filename = match.group()
        source_file = r'C:\CODE\Demo\static\{}'.format(filename)  # Using the filename variable
        shutil.move(r'C:\CODE\Demo\{}'.format(filename),r'C:\CODE\Demo\static\collectionimagess')
        print('**********************************')
        print(source_file)
        image_url =url_for('static',filename= r'collectionimagess/'+filename)
    print(image_url)
    end_time=time.time()
    elapsed_time = end_time - start_time
    print(f"Time consumed for processing request: {elapsed_time} seconds")
    

    return jsonify({'response': df_a,'image_path':image_url})

if __name__ == '__main__':
    app.run(debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [07/May/2024 10:41:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/May/2024 10:41:27] "GET /static/logo.jpg HTTP/1.1" 200 -


blk


c:\Users\ansdwiv\AppData\Local\miniconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
c:\Users\ansdwiv\AppData\Local\miniconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df['production_date'] = pd.to_datetime(df['production_date'])\n\nyear_2022_cars = df[df['production_date'].dt.year == 2022]['model_series'].unique()\n\nyear_2022_cars"}`


NameError: name 'pd' is not defined
Invoking: `python_repl_ast` with `{'query': "import pandas as pd\n\ndf['production_date'] = pd.to_datetime(df['production_date'])\n\nyear_2022_cars = df[df['production_date'].dt.year == 2022]['model_series'].unique()\n\nyear_2022_cars"}`
responded: It seems that the `pd` module is not imported. Let me import it and try again.

['S-Class 223' 'GLC 253' 'C-Class 206' 'A-Class 177' 'GLA 247' 'CLA 118'
 'E-Class 213' 'GLE 167' 'GLB 247' 'B-Class 247' 'C-Class 205'
 'E-Class 238' 'GLS 167' 'CLS-Class 257' 'G-Class 463' 'X 290 AMG']
Invoking: `python_repl_ast` with `{'query': "import matplotlib.pyplot as plt\n\nyear_2022_cars = df[df['production_date'].dt.year == 2022]['model_series'].value_counts()\n\n

[2024-05-07 10:41:55,904] ERROR in app: Exception on /process [POST]
Traceback (most recent call last):
  File "c:\Users\ansdwiv\AppData\Local\miniconda3\Lib\shutil.py", line 825, in move
    os.rename(src, real_dst)
FileNotFoundError: [WinError 2] The system cannot find the file specified: 'C:\\CODE\\Demo\\cars_produced_2022.png' -> 'C:\\CODE\\Demo\\static\\collectionimagess'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\ansdwiv\AppData\Local\miniconda3\Lib\site-packages\flask\app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ansdwiv\AppData\Local\miniconda3\Lib\site-packages\flask\app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ansdwiv\AppData\Local\miniconda3\Lib\site-packages\flask\app.py", line 1823, in full_dispatch_request
    r

I have created a bar chart showing the count of cars produced in the year 2022. The chart has been saved as "cars_produced_2022.png". Here is the chart:

![Cars Produced in 2022](cars_produced_2022.png)

> Finished chain.
what are the different cars produced in the year 2022? give me in png format?
#########df_a#####
 I have created a bar chart showing the count of cars produced in the year 2022. The chart has been saved as "cars_produced_2022.png". Here is the chart:

![Cars Produced in 2022](cars_produced_2022.png)
here is the match if found: <regex.Match object; span=(109, 131), match='cars_produced_2022.png'>
